<a href="https://colab.research.google.com/github/gandhisamay/DSC_recruitment_tasks/blob/main/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
%cd '/content/drive/MyDrive/Datasets '

/content/drive/MyDrive/Datasets 


In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'

--2021-05-27 14:03:19--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [application/x-httpd-php]
Saving to: ‘iris.data’

iris.data           100%[===================>]   4.44K  --.-KB/s    in 0s      

2021-05-27 14:03:19 (48.7 MB/s) - ‘iris.data’ saved [4551/4551]



In [ ]:
df = pd.read_csv('iris.data')

In [ ]:
df

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
...,...,...,...,...,...
144,6.7,3.0,5.2,2.3,Iris-virginica
145,6.3,2.5,5.0,1.9,Iris-virginica
146,6.5,3.0,5.2,2.0,Iris-virginica
147,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
df['Iris-setosa'].value_counts()

Iris-versicolor    50
Iris-virginica     50
Iris-setosa        49
Name: Iris-setosa, dtype: int64

In [ ]:
#Replace the flowers with their one-hot encodes
flowers = {
    'Iris-virginica': 0,
    'Iris-versicolor': 1,
    'Iris-setosa': 2
}
df['Iris-setosa'].replace(flowers,inplace=True)

In [ ]:
#Load the data into numpy arrays
x = np.array(df.loc[:,'5.1':'0.2'])
y = np.array(df['Iris-setosa'])

In [ ]:
#Reshape the arrays
x = x.reshape(-1,4)
y = y.reshape(-1,1)

In [ ]:
#Make train and test datasets
n_test = 30
shuffled_indices = np.random.permutation(df.shape[0])

x_train = x[shuffled_indices[n_test:]]
y_train = y[shuffled_indices[n_test:]]

x_test = x[shuffled_indices[:n_test]]
y_test = y[shuffled_indices[:n_test]]

In [ ]:
x_train.shape

(119, 4)

In [ ]:
#Define the distance function
def euclideanDistance(x,value):
  '''value : Set for which we want to calculate the distance'''
  square_distance = np.sum(np.power((value - x),2),axis=1)
  return np.sqrt(square_distance)

In [ ]:
per_class_predictions = [0,0,0]
for k in range(0,50):
  val = y_train[np.argsort(euclideanDistance(x_train,x_test[0])) == k].item()
  per_class_predictions[val]+=1

In [ ]:
#Define the predictor function 
def predict(euclidDistance,y,k):
  per_class_predictions = [0,0,0]
  flower_array = np.array([0,1,2])
  arg_sort = np.argsort(euclidDistance)
  for k in range(0,k):
    val = y[np.argsort(euclidDistance) == k].item()
    per_class_predictions[val] += 1
  final_sort = np.argsort(np.array(per_class_predictions))
  return flower_array[(np.argsort(np.array(per_class_predictions)) == 2)].item()

In [ ]:
predictions = []
for i in x_test:
  distance = euclideanDistance(x_train,i)
  predictions.append(predict(distance,y_train,k=30))

In [ ]:
#Calculate the accuracy
preds = (np.array(predictions).reshape(30,1) == y_test).sum()
print("Accuracy of model: {}".format(100*preds/y_test.shape[0]))

Accuracy of model: 53.333333333333336


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
preds_linear = model.predict(x_test)

In [ ]:
#Calculate accuracy
preds_final = np.full_like(preds_linear,0)
preds_final[preds_linear < 0.5] = 0 
preds_final[(preds_linear < 1.5)*(preds_linear>0.5)] = 1
preds_final[preds_linear > 1.5] = 2
print("Accuracy of model: {}".format(100*np.sum((preds_final==y_test))/y_test.shape[0]))

Accuracy of model: 93.33333333333333
